In [2]:
import sqlite3
import pandas as pd
from wbe_odm import utilities
pd.options.mode.chained_assignment = 'raise'

In [ ]:
merged = pd.read_csv("../merge_test/merged.csv")
site_measure = pd.read_csv("../merge_test/site_measure.csv")
for df in [merged, site_measure]:
    for col in df.columns:
        if "Unnamed" in col:
            df.drop(columns=[col], inplace=True)
        if "date" in col:
            df[col] = pd.to_datetime(df[col])
        if "collection" in col:
            df[col] = df[col].astype(str).fillna("")
# clean grab dates
merged = utilities.clean_grab_datetime(merged)
# clean composite dates
merged = utilities.clean_composite_data_intervals(merged)

def combine_site_measure(merged, site_measure):
    with sqlite3.connect(':memory:') as conn:
        # write the tables
        merged.to_sql('merged', conn, index=False)
        site_measure.to_sql("site_measure", conn, index=False)
        # write the query
        qry_left = """
            select * from site_measure
            left join merged
            on [SiteMeasure.dateTime] >= [Calculated.dateTimeStart] 
            and [SiteMeasure.dateTime] < [Calculated.dateTimeEnd]
            and [SiteMeasure.siteID] = [Sample.siteID]"""
        qry_right = """
            select * from merged
            left join site_measure
            on [SiteMeasure.dateTime] >= [Calculated.dateTimeStart] 
            and [SiteMeasure.dateTime] < [Calculated.dateTimeEnd]
            and [SiteMeasure.siteID] = [Sample.siteID]"""

        left = pd.read_sql_query(qry_left, conn)
        right = pd.read_sql_query(qry_right, conn)
    return pd.concat([left, right]).drop_duplicates()

#merged = combine_site_measure(merged, site_measure)
# join site measures based on clean dates and site ID
# join cphd based on
#   the smallest polygon for which I have public health data and
#   which contains the site

In [ ]:
cphd = pd.read_csv("../merge_test/cphd_raw.csv", parse_dates=True)
cphd = cphd.add_prefix("CPHD.")
poly = pd.read_csv("../merge_test/poly.csv")
poly


In [ ]:
def has_cphd_data(x, uniques):
    if pd.isna(x):
        return None
    ls = x.split(";")
    ls = [i for i in ls if i in uniques]
    return ";".join(ls) if ls else None

def pick_cphd_poly(x, poly):
    if pd.isna(x):
        return None
    ls = x.split(";")
    areas = []
    for id_ in ls:
        area = poly.loc[poly["Polygon.polygonID"] == id_, "area"]
        areas.append(area)
    min_area = min(areas)
    min_idx = areas.index(min_area)
    return ls[min_idx]


def smallest_area_with_cphd(merged, poly, cphd):
    poly["shape"] = poly["Polygon.wkt"].apply(lambda x: utilities.convert_wkt(x))
    poly["area"] = poly["shape"].apply(lambda x: x.area)
    unique_cphd_polys = cphd["CPHD.polygonID"].unique()
    merged["poly_w_cphd"] = merged["polygonIDs"].apply(lambda x: has_cphd_data(x, unique_cphd_polys))
    merged["poly_for_cphd"] = merged["poly_w_cphd"].apply(lambda x: pick_cphd_poly(x, poly))
    poly.drop(columns=["shape", "area"], inplace=True)
    merged.drop(columns=["Calculated.cphd_poly"], inplace=True)
    return merged

def combine_cphd_w_samples_w_polygons(merged, cphd, poly):
    merged = smallest_area_with_cphd(merged, poly, cphd)

    with sqlite3.connect(':memory:') as conn:
        # write the tables
        merged.to_sql('merged', conn, index=False)
        cphd.to_sql("cphd", conn, index=False)
        # write the query
        qry_left = """
            select * from cphd
            left join merged
            on [CPHD.date] >= [Calculated.dateTimeStart]
            and [CPHD.date] < [Calculated.dateTimeEnd]
            and [Calculated.cphd_poly] = [CPHD.polygonID]
            """
        qry_right = """
            select * from merged
            left join cphd
            on [CPHD.date] >= [Calculated.dateTimeStart]
            and [CPHD.date] < [Calculated.dateTimeEnd]
            and [Calculated.cphd_poly] = [CPHD.polygonID]
            """

        left = pd.read_sql_query(qry_left, conn)
        right = pd.read_sql_query(qry_right, conn)
    return pd.concat([left, right]).drop_duplicates()


In [ ]:
unique_cphd_polys = cphd["polygonID"].unique().to_list()

In [6]:
import plotly.express as px

In [70]:
df = pd.read_csv("../testA.csv", na_values="na", parse_dates=True)
df["Calculated.timestamp"] = pd.to_datetime(df["Calculated.timestamp"])
df["Calculated.timestamp"].dtype


dtype('<M8[ns]')

In [77]:
def build_site_specific_dataset(df, site_id):
    filt_site1 = df["Site.siteID"] == site
    filt_site2 = df["SiteMeasure.siteID"] == site
    filt_site = filt_site1 | filt_site2
    df1 = df[filt_site]

    cphd_poly_id = str(
        df.loc[filt_site1, "Calculated.polygonIDForCPHD"].iloc[0]).lower()
    poly_filt = df["CPHD.polygonID"].fillna("").str.lower().str.match(cphd_poly_id)
    df2 = df[poly_filt]
    dataset = pd.concat([df1, df2], axis=0)
    dataset = dataset.set_index(["Calculated.timestamp"])
    dataset.sort_index()
    return dataset.reindex(sorted(dataset.columns), axis=1)

In [ ]:
test = build_site_specific_dataset(df, "qc_01")
cols = [col for col in test.columns if "value" in col]
for col in cols:
    fig = px.scatter(test[cols], y=col)
    fig.show()

,Sample.sampleID,Sample.siteID,Sample.instrumentID,Sample.reporterID,Sample.dateTime,Sample.dateTimeStart,Sample.dateTimeEnd,Sample.type,Sample.collection,Sample.preTreatment,...,Sewershed.Polygon.wkt,Sewershed.Polygon.link,Sewershed.Polygon.notes,CPHD.cphdID,CPHD.reporterID,CPHD.polygonID,CPHD.date,CPHD.notes,CPHD.death_report_value,CPHD.conf_report_value
9,qc_01_cptp24h_rawwwww_2021-04-15_1,qc_01,NaN,city_qc,NaN,2021-04-14 00:00:00,2021-04-15 00:00:00,rawwwww,cptp24h,True,...,"POLYGON ((-71.252979 46.9156699,-71.2635805 46...",https://mcgill.sharepoint.com/:f:/r/sites/mice...,this polygon encompasses the entire sewer netw...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,qc_01_cptp24h_rawww_2021-02-12_1,qc_01,NaN,city_qc,NaN,2021-02-11 00:00:00,2021-02-12 00:00:00,rawww,cptp24h,True,...,"POLYGON ((-71.252979 46.9156699,-71.2635805 46...",https://mcgill.sharepoint.com/:f:/r/sites/mice...,this polygon encompasses the entire sewer netw...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,qc_01_cptp24h_rawww_2021-04-14_1,qc_01,NaN,city_qc,NaN,2021-04-13 00:00:00,2021-04-14 00:00:00,rawww,cptp24h,True,...,"POLYGON ((-71.252979 46.9156699,-71.2635805 46...",https://mcgill.sharepoint.com/:f:/r/sites/mice...,this polygon encompasses the entire sewer netw...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,qc_01_cptp24h_rawww_2021-01-24_1,qc_01,NaN,city_qc,NaN,2021-01-23 00:00:00,2021-01-24 00:00:00,rawww,cptp24h,True,...,"POLYGON ((-71.252979 46.9156699,-71.2635805 46...",https://mcgill.sharepoint.com/:f:/r/sites/mice...,this polygon encompasses the entire sewer netw...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,qc_01_cptp24h_rawww_2021-01-15_1,qc_01,NaN,city_qc,NaN,2021-01-14 00:00:00,2021-01-15 00:00:00,rawww,cptp24h,True,...,"POLYGON ((-71.252979 46.9156699,-71.2635805 46...",https://mcgill.sharepoint.com/:f:/r/sites/mice...,this polygon encompasses the entire sewer netw...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,inspq_capitale-nationale_conf_report_2021-03-28,inspq,prov_qc_hlthreg_capitale-nationale,2021-03-28,NaN,NaN,114.0
2864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,inspq_capitale-nationale_conf_report_2021-03-01,inspq,prov_qc_hlthreg_capitale-nationale,2021-03-01,NaN,NaN,36.0
2878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,inspq_capitale-nationale_conf_report_2020-12-28,inspq,prov_qc_hlthreg_capitale-nationale,2020-12-28,NaN,NaN,175.0
2968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,inspq_capitale-nationale_conf_report_2021-01-21,inspq,prov_qc_hlthreg_capitale-nationale,2021-01-21,NaN,NaN,79.0


qc_01_swrcat prov_qc_hlthreg_capitale-nationale


In [8]:
len(df)

2927

1409
1360
1015
1130


In [19]:
df.dropna(subset=["Calculated.plotDate", "SiteMeasure.dateTime"])[["Calculated.plotDate", "SiteMeasure.dateTime"]]

,Calculated.plotDate,SiteMeasure.dateTime
294,2021-04-05 12:00:00,2021-04-05 03:50:00
295,2021-04-05 12:00:00,2021-04-05 00:20:00
296,2021-04-05 12:00:00,2021-04-05 06:35:00
297,2021-04-05 12:00:00,2021-04-05 09:05:00
298,2021-04-05 12:00:00,2021-04-05 12:55:00
...,...,...
2053,2021-04-08 12:00:00,2021-04-08 05:00:00
2056,2021-04-08 12:00:00,2021-04-08 12:30:00
2057,2021-04-06 12:00:00,2021-04-06 12:30:00
2058,2021-04-15 12:00:00,2021-04-15 11:10:00


In [ ]:
def build_dataset_for_site()